# Deploying Python code to servers

<blockquote style="font-family: 'Times New Roman', serif; font-size: 20px; font-style: italic;"><span style="font-size: 4rem;">T</span>here are things that are hard, and then there's Python deployment. &mdash; me</blockquote>

So you have some Python code that you would like to run on the web? This can become staggeringly more difficult than you might expect. First, let's be clear what we're talking about.

- deploy *desktop software* to users' computers and devices
- deploy *server software* to a server, typically on cloud infrastructure like Amazon Web Services (AWS) or Google Cloud Platform, Heroku, PythonAnywhere and so on

In this essay, we are focusing only on the second one, i.e., deployment of Python code to **servers**.

There *are* easy ways to do this: [Google App Engine](https://cloud.google.com/appengine/docs) makes it almost completely painless, **as long as** you [stick to Python 2.7](http://stackoverflow.com/a/24229338/170656) and [avoid packages with C-extensions](https://cloud.google.com/appengine/docs/python/runtime#Python_Pure_Python) 

I'm kinda into [Cython](http://cython.org/) so this is a problem for me, never mind the requirement for legacy Python. So what to do?

## Web Hosts, Web Hosts everywhere

There are many places to host Python code online. [PythonAnywhere](https://www.pythonanywhere.com/) is a top-notch solution that makes it very, very easy to get going with your Python app, **as long as** your Python version is one of 2.6, 2.7, 3.3 and 3.4. Since there is shell access, you *can* also use virtualenv, but I'm grouping PythonAnywhere in the "does a lot for you" camp because they have a cool UI that simplifies much of the server administration tasks you would normally need to do from the shell.

[Heroku]() is the other popular choice, **as long as** the Python version you need happens to [match one of the two supported runtimes](https://devcenter.heroku.com/articles/python-runtimes#supported-python-runtimes).

Heroku does auto-scaling (just like App Engine), but with PythonAnywhere you just need to switch plans for more resources.

## Moving towards server administration AKA What fresh, new hell is this?

If the restrictions are too much, you might have to set up your own server, like an Amazon EC2 instance, or a Rackspace/Linode server, and many others.  One of the best places to do that is [DigitalOcean](https://www.digitalocean.com/community/tutorials/how-to-set-up-ubuntu-cloud-servers-for-python-web-applications), not least because their guides are *excellent*. The guides do also work well at other hosts though.  

What you're pretty much doing at this point is taking on the role of a system administrator, likely *without any formal training*, and even more likely **without an awareness of the Linux distribution life cycle and support practices**, particularly if your development platform is OS X or Windows.  Unfortunately, it turns out that if you don't make an effort to understand those upstream issues, you subject yourself to surprising and painful experiences, like that time when [Ubuntu just downgraded its Python 3 package in the Long Term Support version, without fixing packages that depended on that version leaving my docker deployment in a broken state](https://bugs.launchpad.net/ubuntu/+source/python3.4/+bug/1503382). (I *chose* the LTS specifically to avoid things like that!)  Don't worry, eventually it all got fixed a few weeks later.

But let's get back to the deployment of Python apps.

## Patterns, Anti-patterns - Can you tell the difference?

If you sink a lot of time into reading blogs and StackOverflow, there is much wisdom to be found. [Hynek](https://hynek.me/articles/python-deployment-anti-patterns/) made an excellent post about deployment anti-patterns to avoid.

[Graham Dumpleton](http://blog.dscpl.com.au/2016/01/python-virtual-environments-and-docker.html) has an extensive blog series going about deploying your Python code in Docker containers, and the nature of the content is very much a collection of good patterns, and anti-patterns to avoid.  I've read them several times, and still don't quite get everything, especially with respect to Docker, but I *feel* like if just read through it one more time, all will become clear :/

# ...which brings us to Docker

In case you don't know, `Docker` is a very clever tool that makes it easy for you run virtual machines. The docker people don't like to hear that, but honestly it's the easiest mental picture until you need to care about the details.  The easiness has direct implications for two things:
- Easy to set up locally, to make development easier
- Easy to re-create the virtual machine on a production server, making deployment easier

### You still have to do system administration!  Docker doesn't solve this!

You *still* have to do system administration; `docker` just makes it easy to work with a "known" configuration.  This means that it doesn't matter whether your project runs on Amazon Web Services, or Openshift, or Google Cloud Platform: you can deploy the same docker image to all of them.

However, **this doesn't solve all our problems**.

# What are our problems?

Imagine the following:
- We want to deploy a service onto a single server
- This service uses several different applications locally.  
- Each application is a Python application
- Each Python application uses one or more versions of **Python**
- Each Python application uses a different set of packages, each pinned themselves to different versions

The tricky part is *one or more versions of Python*.

## Different Versions of Python

Linux distros tend to be slow-moving, for the obvious reason that updating packages requires an enormous coordination effort to make sure that upgrading one thing doesn't break other things.  Unfortunately, this means that it can be difficult to use version of software, say, Python, that are not the *official packaged versions* of the Linux distribution you're targetting.

However, this *isn't the actual problem*.  It doesn't matter how fast the distro moves.  The issue is really about being able to pin software versions to a specific version that you choose.  And only upgrade *when* you want.

## About virtualenv - version pinning

`virtualenv` isolates different Python applications' dependecies from each other.  For example, suppose you have one Python application that needs Numpy 1.7.1 and a different Python application that needs Numpy 1.8. `virtualenv` will allow you to install and run both Python applications on the same machine.  However, you are still restricted by the Python interpreter running on that machine.  On most Linux distributions (at the time of writing), you can install Python 2.7 and Python 3.4, but not 3.5.

Assuming you installed both 2.7 and 3.4, you could make virtualenvs of either of them, but you **cannot make a virtualenv for 3.5**.  Virtualenv doesn't work like that.

In the future, popular linux distributions will likely upgrade 3.4 to 3.5.  **This does not solve our problems**.

## Version pinning - also for Python itself

The entire point of using virtualenv in the first place, is to allow us to pin our app dependencies and isolate the **version** requirements of different apps from each other. What we *really* want, is to have version-pinning **also for Python itself**.  

Let's imagine we have two Python apps, `alpha` and `beta`.  The first, `alpha` was written against 2.7, while the second, `beta` was written against 3.4.  Because they used different versions of Python, *strictly speaking* we don't need to use virtualenv, but because we fear the wrath of Hynek we did it anyway.

It was just as well: six months down the road, a new app is deployed to the same server, `gamma`, which also runs on 3.4, but which uses different versions of some of the app dependencies.  For example, suppose `beta` uses Numpy 1.7.1 while `gamma` uses Numpy 1.8.  Virtualenv allows us to keep `beta` and `gamma` running on their own dependencies, even though they are both using 3.4

*Now* suppose that we deploy a new app, `delta` to the same server, but that app needs to run on Python 3.5!  What we **don't want to do** is have to upgrade `beta` and `gamma` also to 3.5.  For one thing, there may be some packages that are dependencies of `beta` and `gamma` that have not yet been made available for 3.5.

# How to do version pinning for <u>Python</u> itself

Right now, as far as I am aware there are two main routes:
0. [Conda](http://conda.pydata.org/docs/)
0. [pyenv](https://github.com/yyuu/pyenv)

There is another route, called *Compiling Python from source yourself* which I won't go into here.  If you're going to do it all by hand then you will inevitably have to start building the same kinds of tools that the other routes already provide.

## Conda

Conda is a tool that you normally obtain from, and use with the [Anaconda Python](https://www.continuum.io/downloads) distribution from Continuum Analytics.  **`conda` is both a package manager and a virtual environment manager**.  This means that not only do you install Python packages with it, but you can also create new Python virtual environments, just like virtualenv. There are other features of the conda ecosystem, such as the support for built (compiled) packages so that you don't need to compile packages yourself like you would have to do with `pip`.

(You *can* install built packages with `pip` if there is a `wheel` available for that package, however not all packages provide wheels.  With `conda` almost all the popular packages are available in a pre-compiled, or "built", format.)

What we are interested in is the support for virtual environments.  Using `conda` you can install and activate different versions of Python from 2.6 to 3.5, *and* you can create multiple virtual environments off each of those.

One of the niggles with using `conda` is that **not all Python packages** are available in the special `conda` repository; `conda` does not install from the Python Package Index, but rather its own repository that is maintained by Continuum Analytics.  The fallback, in such situations, is to use `pip` from inside the `conda` virtual environment and install extra packages that way. Now you have to deal with a chimeric environment where there are two package managers competing for control (within that environment).  They *mostly* get along when you're running all the commands manually, but we're interested in **servers** and **automated deployment**.

Which one must use `requirements.txt`?  `conda` or `pip`?  This difficult question, and similar issues, is why I generally don't deploy to servers using `conda`.  It makes things more complicated, including the explanations you have to give to other devs on the project about what `conda` even is.

## pyenv

Pyenv is an *even more clever hack* than what `virtualenv` does, and `pyenv` is really closer to what `conda` is doing for virtual environments.  Note that `pyenv` is not a package manager though: it only deals with Python environments, which means you would still use `pip` for installing packages.  Also, `pyenv` is not a replacement for `virtualenv` (which makes *virtual* Python environments), but it can work with `virtualenv`.  Confused yet?

Pyenv is a tool for installing multiple different Python versions, including not only the official CPython releases, but even PyPy versions and even [Stackless Python](http://stackless.readthedocs.org/en/latest/stackless-python.html).

Unfortunately, `pyenv` does not work on Windows (`conda` does).

# Bring on the pain AKA a demonstration with pyenv and Docker

# Docker

Getting docker set up is well beyond the scope of this post.  I'm using docker only as a very simple way to start a VM with the Centos Linux distribution. (Perhaps I should make another blog post called "Docker for ordinary devs"...)

You don't need to use docker for this.  You can also just start a VM using VMware or Virtualbox.  However, to get going with docker, once it is set up, is quite easy:

```shell
docker run -i -t centos:7.2.1511 /bin/bash
```

And then you'll see the prompt change to something like this:

```shell
[root@7bc15f9e0045 /]#
```

In the following steps, I change the prompt to `$` to keep it simpler (and more general).


# Setting up `pyenv`

Install the development tools (this is specific to the Centos and Red Hat distributions of Linux):

```bash
$ yum groupinstall 'Development Tools'
```

Certain libraries are needed to build Python, which we are going to do a bunch of times:

```bash
$ yum install zlib-devel bzip2 bzip2-devel readline-devel sqlite sqlite-devel openssl-devel
```

Make a folder for the upcoming pydev installer:

```bash
$ mkdir pyenv
$ cd pyenv
```

Obtain the pyenv installer (copied from the pyenv docs):

```bash
$ curl -L https://raw.githubusercontent.com/yyuu/pyenv-installer/master/bin/pyenv-installer | bash
```

This will fail because you also need `git`:

```
$ yum install git
```

Now do the `curl` command again.

Need to add some things to your bash profile:

```
$ echo 'export PYENV_ROOT="$HOME/.pyenv"' >> ~/.bash_profile
$ echo 'export PATH="$PYENV_ROOT/bin:$PATH"' >> ~/.bash_profile
```

and

```shell
$ echo 'eval "$(pyenv init -)"' >> ~/.bash_profile
```

Now re-read your `bash` config file to set up pyenv:

```bash
$ source ~/.bash_profile
```

Install a Python version:

```
$ pyenv install 2.7.11
```

Note that this doesn't automatically "activate" or do anything with the new Python, it just installs it. You can check to see which Python is currently active:

```bash
$ pyenv version
system (set by /root/.pyenv/version)
```

You can set up things so that a specific Python version is associated with a **particular directory**.  This is called the **`local`** setting.  You can also change out the **`global`** python interpreter using the subcommand of the same name. Lastly, you can change the **`shell`** interpreter.  Let's do that by changing from the default to the one we just installed.

```shell 
$ pyenv versions
* system (set by /root/.pyenv/version)
  2.7.11
$ python -V
Python 2.7.5
$ pyenv shell 2.7.11
$ python -V
Python 2.7.11
```

The `2.7.5` is the default Python installed on this version of Centos, 7.2.1511.

Let's add another Python:

```shell
$ pyenv install 3.5.1
sha256sum: the --quiet option is meaningful only when verifying checksums
Try 'sha256sum --help' for more information.
Downloading Python-3.5.1.tgz...
-> https://www.python.org/ftp/python/3.5.1/Python-3.5.1.tgz
Installing Python-3.5.1...
Installed Python-3.5.1 to /root/.pyenv/versions/3.5.1
```

Listing out the versions shows that 3.5.1 is now available:

```shell
$ pyenv versions
  system
* 2.7.11 (set by PYENV_VERSION environment variable)
  3.5.1
```

It's also pretty great how the feedback tells us that the currently active one was set for the shell.

So now we have decoupled Python versions from the operating system. In fact we can now install **any of these Python versions** in this incredibly long list (apologies for the length; just scroll past it if you like):

```
$ pyenv install --list
Available versions:
  2.1.3
  2.2.3
  2.3.7
  2.4
  2.4.1
  2.4.2
  2.4.3
  2.4.4
  2.4.5
  2.4.6
  2.5
  2.5.1
  2.5.2
  2.5.3
  2.5.4
  2.5.5
  2.5.6
  2.6.6
  2.6.7
  2.6.8
  2.6.9
  2.7-dev
  2.7
  2.7.1
  2.7.2
  2.7.3
  2.7.4
  2.7.5
  2.7.6
  2.7.7
  2.7.8
  2.7.9
  2.7.10
  2.7.11
  3.0.1
  3.1
  3.1.1
  3.1.2
  3.1.3
  3.1.4
  3.1.5
  3.2-dev
  3.2
  3.2.1
  3.2.2
  3.2.3
  3.2.4
  3.2.5
  3.2.6
  3.3.0
  3.3-dev
  3.3.1
  3.3.2
  3.3.3
  3.3.4
  3.3.5
  3.3.6
  3.4.0
  3.4-dev
  3.4.1
  3.4.2
  3.4.3
  3.4.4
  3.5.0
  3.5-dev
  3.5.1
  3.6-dev
  anaconda-1.4.0
  anaconda-1.5.0
  anaconda-1.5.1
  anaconda-1.6.0
  anaconda-1.6.1
  anaconda-1.7.0
  anaconda-1.8.0
  anaconda-1.9.0
  anaconda-1.9.1
  anaconda-1.9.2
  anaconda-2.0.0
  anaconda-2.0.1
  anaconda-2.1.0
  anaconda-2.2.0
  anaconda-2.3.0
  anaconda-2.4.0
  anaconda2-2.4.0
  anaconda2-2.4.1
  anaconda3-2.0.0
  anaconda3-2.0.1
  anaconda3-2.1.0
  anaconda3-2.2.0
  anaconda3-2.3.0
  anaconda3-2.4.0
  anaconda3-2.4.1
  ironpython-dev
  ironpython-2.7.4
  ironpython-2.7.5
  jython-dev
  jython-2.5.0
  jython-2.5-dev
  jython-2.5.1
  jython-2.5.2
  jython-2.5.3
  jython-2.5.4-rc1
  jython-2.7.0
  jython-2.7.1b1
  jython-2.7.1b2
  miniconda-latest
  miniconda-2.2.2
  miniconda-3.0.0
  miniconda-3.0.4
  miniconda-3.0.5
  miniconda-3.3.0
  miniconda-3.4.2
  miniconda-3.7.0
  miniconda-3.8.3
  miniconda-3.9.1
  miniconda-3.10.1
  miniconda-3.16.0
  miniconda-3.18.3
  miniconda2-latest
  miniconda2-3.18.3
  miniconda2-3.19.0
  miniconda3-latest
  miniconda3-2.2.2
  miniconda3-3.0.0
  miniconda3-3.0.4
  miniconda3-3.0.5
  miniconda3-3.3.0
  miniconda3-3.4.2
  miniconda3-3.7.0
  miniconda3-3.8.3
  miniconda3-3.9.1
  miniconda3-3.10.1
  miniconda3-3.16.0
  miniconda3-3.18.3
  miniconda3-3.19.0
  pypy-c-jit-latest
  pypy-c-nojit-latest
  pypy-dev
  pypy-stm-2.3
  pypy-portable-2.3.1
  pypy-portable-2.4
  pypy-portable-2.5
  pypy-portable-2.5.1
  pypy-stm-2.5.1
  pypy-portable-2.6
  pypy-portable-2.6.1
  pypy-portable-4.0
  pypy-portable-4.0.1
  pypy-1.5-src
  pypy-1.5
  pypy-1.6
  pypy-1.7-dev
  pypy-1.7
  pypy-1.8-dev
  pypy-1.8
  pypy-1.9-dev
  pypy-1.9
  pypy-2.0-dev
  pypy-2.0-src
  pypy-2.0
  pypy-2.0.1-src
  pypy-2.0.1
  pypy-2.0.2-src
  pypy-2.0.2
  pypy-2.1-src
  pypy-2.1
  pypy-2.2-src
  pypy-2.2
  pypy-2.2.1-src
  pypy-2.2.1
  pypy-2.3-src
  pypy-2.3
  pypy-2.3.1-src
  pypy-2.3.1
  pypy-2.4.0-src
  pypy-2.4.0
  pypy-2.4-beta1-src
  pypy-2.4-beta1
  pypy-2.5.0-src
  pypy-2.5.0
  pypy-2.5.1-src
  pypy-2.5.1
  pypy-2.6.0-src
  pypy-2.6.0
  pypy-2.6.1-src
  pypy-2.6.1
  pypy-4.0.0-src
  pypy-4.0.0
  pypy-4.0.1-src
  pypy-4.0.1
  pypy3-dev
  pypy3-portable-2.3.1
  pypy3-portable-2.4
  pypy3-2.3.1-src
  pypy3-2.3.1
  pypy3-2.4.0-src
  pypy3-2.4.0
  stackless-dev
  stackless-2.7-dev
  stackless-2.7.2
  stackless-2.7.3
  stackless-2.7.4
  stackless-2.7.5
  stackless-2.7.6
  stackless-2.7.7
  stackless-2.7.8
  stackless-3.2-dev
  stackless-3.2.2
  stackless-3.2.5
  stackless-3.3-dev
  stackless-3.3.5
  stackless-3.4.1
```

However, we **still have a problem** because different Python apps using the **same version of Python** still need version pinning for their respective dependencies.  Remember the example of one app needing Numpy 1.7.1 and the other needing Numpy 1.8?

This is what **virtualenv** is for, and you can also use it with `pyenv`:

```shell
$ pyenv versions
  system
* 2.7.11 (set by PYENV_VERSION environment variable)
  3.5.1
```

Now lets create a new `virtualenv` for the 3.5.1 version of Python:

```shell
$ pyenv virtualenv 3.5.1 my_env_35
Ignoring indexes: https://pypi.python.org/simple
Requirement already satisfied (use --upgrade to upgrade): setuptools in /root/.pyenv/versions/3.5.1/envs/my_env_35/lib/python3.5/site-packages
Requirement already satisfied (use --upgrade to upgrade): pip in /root/.pyenv/versions/3.5.1/envs/my_env_35/lib/python3.5/site-packages
```

Listing out the available versions again:

```shell
pyenv versions
  system
* 2.7.11 (set by PYENV_VERSION environment variable)
  3.5.1
  3.5.1/envs/my_env_35
  my_env_35
```

The virtualenv is listed separately, and twice.  I don't know why.

You can activate the new virtualenv in two ways:

```shell
$ pyenv shell my_env_35
pyenv-virtualenv: deactivate
pyenv-virtualenv: activate my_env_35
pyenv-virtualenv: prompt changing will be removed from future release. configure `export PYENV_VIRTUALENV_DISABLE_PROMPT=1' to simulate the behavior.
(my_env_35) $ python -V
Python 3.5.1
```

This method treats the virtualenv like just another Python instance, which is nice. To do the deactivation, just switch to another Python version:

```shell
$ pyenv shell system
pyenv-virtualenv: deactivate 3.5.1/envs/my_env_35
$ python -V
Python 2.7.5
```

The other way is to use `pyenv activate <envname>` and `pyenv deactivate`.

# Running your app

The easiest way is probably to create a short [wrapper shell script](http://stackoverflow.com/a/30222231/170656) to change to the correct env, and then call your app:

```shell
# run.sh
pyenv shell 3.5.1
python myapp.py
```

And then the way you start your app becomes:

```shell
$ source run.sh
```

(You can also put a bash shebang at the top of `run.sh` and make it executable, in the usual way.)

If you have multiple apps, this becomes tedious, so rather just have a single runner, and pass it args:

```shell
# run.sh
pyenv shell 3.5.1
python "$@"
```

The `"$@"` arguments is special (and [make sure you include the quotes](http://stackoverflow.com/a/3816747/170656)). It makes sure all the arguments passed to your `run.sh` script will get passed on.  So now the name of your Python app becomes an argument to run.sh:

```shell
$ source run.sh myapp.py
(myapp.py runs under 3.5.1)
$ source run.sh myotherapp.py
(myotherapp.py also runs 3.5.1)
```

If you're still paying attention and have not been completely exhausted yet, it should be easy to see how you might generalize `run.sh` so that you can also pass the name of the Python environment to use. However, it begins to get a bit fiddly with managing the list of command-line parameters if some of them (e.g. "3.5.1") are not meant for the python script, but you still want to pass everything _else_ to your app.  

In this case, it will be much easier to simply set an environment variable.

### In fact, `pyenv` already provides this by default

You don't even need to muck around with a shell script at all.  To have exactly the same effect as running `pyenv shell 3.5.1`, all you need is this:

```shell
$ PYENV_VERSION=my_env_35 python myapp.py
(myapp.py runs under the virtualenv)
$ PYENV_VERSION=system python myapp.py
(myapp.py runs under the system python)
$ PYENV_VERSION=2.7.11 python myapp.py
(myapp.py runs under Python 2.7.11)
```